# Harmonic similarity demo

**Notes for improvements**
- For computational reasons, it is better to start comparing n-grams from the lowest order (e.g. 2, 3) and stop as soon as there are no more elements in common. For instance, if there are no repeated patterns of len 4 between two sequences under comparison, it does not make sense to try  with anything > 4. This can speed up computation of the hsim.
- Instead of saving the actual full ngrams for each track, we should consider creating a table of all the possible ngrams that were found across all the dataset. This  way track A will say "I have ngrams x, y, ..." which can then be read from such a different table. This would reduce the space complexity.
- In addition, to speed up comparison, we should create another table -- using the indices of the previous table (the index for all the ngrams), to save all those tracks that each specific ngram. The mapping is <from the index of the ngram> to the list of all the track (ids) that have that recurring pattern. This should easily allow comparing 1 given track to all the others in one shot, rather than performing trivial 1-to-1 comparisons.
- The current code works on ngrams computed from tracks that were **not** transposed to the same key (considering the current encodings), so the harmonic similarity should be much higher, in principle, once we manage to address this point.
- The second term h_corr of our full measure of harmonic similarity (see Notion) is the most computationally demanding part, because it requires re-computing the recurring patterns from the concatenated sequences. Not supported for the moment, but we can consider as FW, as we will be able to retrieve more matches and increase the whole similarity score.
- **Doing graph analysis of the resulting graph (see the Extra section) seems a very very promising direction!**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import joblib
import matplotlib.pyplot as plt

from harmonic_lib import ngram_hsim

## Reading and re-shaping the data

In [3]:
with open("../setup/sonar_ngrams_global.joblib", "rb") as fo:
    ngrams_bag_raw = joblib.load(fo)

with open("../setup/sonar_encoding_bundle.joblib", "rb") as fo:
    data_bundle = joblib.load(fo)

ngrams_bag = {list(track_list.keys())[0]: list(track_list.values())[0] \
    for track_list in ngrams_bag_raw}

In [4]:
encdec = data_bundle["encoder_decoder"]

In [5]:
ngrams_bag["isophonics_0"][:10]  # just an example to visualise

[(106, 264, 227),
 (106, 228, 112),
 (227, 10, 106),
 (112, 10, 106),
 (10, 106, 264),
 (228, 112, 10),
 (131, 238, 227),
 (10, 106, 10),
 (291, 131, 238),
 (106, 131, 10)]

## Computing the similarity: an example

In [6]:
a_rpbag = ngrams_bag["isophonics_0"]
b_rpbag = ngrams_bag["isophonics_155"]

In [7]:
hsim_score, longest_rp = ngram_hsim(a_rpbag, b_rpbag)

print("h_sim = {0:.4f} based on longest recurrent pattern(s): {1}"
      .format(hsim_score, longest_rp))

h_sim = 0.0000 based on longest recurrent pattern(s): []


## Computing the similarity: all tracks

In [63]:
track_ids = list(ngrams_bag.keys())
hsim_map = {track_id: {} for track_id in track_ids}

In [64]:
for i, track_a in enumerate(track_ids):
    a_rpbag = ngrams_bag[track_a]  # fix for now
    for j in range(i+1, len(track_ids)):  # move ahead
        track_b = track_ids[j]
        b_rpbag = ngrams_bag[track_b]

        hsim, longest_rps = ngram_hsim(a_rpbag, b_rpbag)
        longest_rps = [[encdec.decode_event(idx) for idx in lsrp_shot] \
            for lsrp_shot in longest_rps]  # keep and decode
        if hsim > 0.:  # save only non-trivial
            hsim_map[track_a][track_b] = hsim, longest_rps
            hsim_map[track_b][track_a] = hsim, longest_rps


In [65]:
#hsim_map["isophonics_0"]["isophonics_155"]

In [66]:
with open("../setup/sonar_hsim_map_global_b.joblib", "wb") as fo:
    joblib.dump({"hsim_map": hsim_map}, fo)

## Extra

In [1]:
import matplotlib.pyplot as plt
import networkx as nx

from bokeh.io import output_notebook, output_file, show, save
from bokeh.models import (Circle, MultiLine, HoverTool, Row, Column, TextInput,
                          NodesAndLinkedEdges, CustomJS, RangeSlider, Div, Button)
from bokeh.plotting import figure, from_networkx
from bokeh.palettes import Greys256, Blues256, Spectral8
from bokeh.transform import linear_cmap

from visualisation import filtration_code, node_info_code

output_notebook()


Loading BokehJS ...

In [7]:
import joblib

with open("../setup/sonar_hsim_map_global.joblib", "rb") as fo:
    hsim_map = joblib.load(fo)["hsim_map"]

# with open("../setup/hsim_exp_2.joblib", "rb") as fo:
#     hsim_map = joblib.load(fo)



In [15]:
# meta_artist, meta_title, meta_dict = {}, {}, {}

# track_ids = list(hsim_map.keys())

In [69]:
import pandas as pd

meta_df = pd.read_csv("../setup/sonar_datasets_meta.csv")
meta_df["link"] = 'https://soundcloud.com/jacopo-de-berardinis/' \
     + meta_df["id"].str.replace("_", "-")  # append sonification
meta_df = meta_df.set_index("id")
meta_dict = meta_df.to_dict("index")

meta_artist = {track_id: track_meta["artist"] \
    for track_id, track_meta in meta_dict.items()}
meta_title = {track_id: track_meta["title"] \
    for track_id, track_meta in meta_dict.items()}

In [10]:
G = nx.Graph()

for track_id in track_ids:
    if len(hsim_map[track_id]) > 0:
        G.add_node(track_id)

for track_a in hsim_map:
    for track_b in hsim_map[track_a]:
        if not G.has_edge(track_b, track_a):
            hsim_val, lsrp = hsim_map[track_a][track_b]
            # Decode the longest shared recurrent pattern
            # lsrp = [[encdec.decode_event(idx) for idx in lsrp_shot] \
            #     for lsrp_shot in lsrp]  # keep all longest shared patterns 
            G.add_edge(track_a, track_b, weight=hsim_val, lsrp=lsrp)

In [11]:
# Some basic network analysis
degrees_dict = dict(nx.degree(G))
nx.set_node_attributes(G, name='degree', values=degrees_dict)

degrees = list(degrees_dict.values())
communities = nx.algorithms.community.greedy_modularity_communities(G)

mod_class, mod_color = {}, {}
# Loop through each community in the network
for community_number, community in enumerate(communities):
    # For each member of the community, add their community number and a distinct color
    for name in community: 
        mod_class[name] = community_number
        mod_color[name] = Spectral8[community_number]

In [12]:
def rescale(val, min_val, max_val, new_range=(1, 20)):
    normalised = ((val - min_val)*(new_range[1] - new_range[0])) / (max_val - min_val)
    return new_range[0] + normalised

min_degree, max_degree = min(degrees), max(degrees)
node_size = dict([(node, rescale(degree, min_degree, max_degree)) \
    for node, degree in nx.degree(G)])

nx.set_node_attributes(G, name='node_size', values=node_size)
# Add modularity class and color as attributes from the network above
nx.set_node_attributes(G, mod_class, 'modularity_cls')
nx.set_node_attributes(G, mod_color, 'modularity_col')
# Add additional information about tracks to the nodes
nx.set_node_attributes(G, meta_title, 'node_title')
nx.set_node_attributes(G, meta_artist, 'node_artist')

In [74]:
dataset_cols = {
    "isophonics": "black",
    "jaah": "red",
    "schubert-winterreise": "yellow"}

dataset_nodes = {node: dataset_cols[node.split("_")[0]] for node in list(G.nodes)}
nx.set_node_attributes(G, dataset_nodes, 'dataset_col')

In [13]:
# dataset_nodes = {node: "black" for node in list(G.nodes)}
# nx.set_node_attributes(G, dataset_nodes, 'dataset_col')

In [16]:
title = 'Harmonic similarity network'

# Colouring options
edge_palette = Greys256[::-1]
node_palette = Blues256[::-1]
# From palettes to colourmaps
edge_cmap = linear_cmap(
    'weight', edge_palette, 0, 1)
node_degree_cmap = linear_cmap(
    "degree", node_palette, min(degrees), max(degrees))

#Create a plot — set dimensions, toolbar, and title
plot = figure(tools="pan,wheel_zoom,save,reset,tap",
              active_scroll='wheel_zoom', title=title, sizing_mode="stretch_both")
# Create a network graph object with spring layout
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

# Set the colour and size of nodes and edges
network_graph.node_renderer.glyph = Circle(
    size="node_size", fill_color="modularity_col", line_color="dataset_col")
network_graph.edge_renderer.glyph = MultiLine(
    line_color=edge_cmap,
    line_alpha=0.5, line_width=1)

# Selection behaviour: change of col/size
network_graph.node_renderer.selection_glyph = Circle(
    size=10, fill_color="white", line_width=.5)
network_graph.edge_renderer.selection_glyph = MultiLine(
    line_color="black", line_width=2)

network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

# Hovering behaviour: show information
hover_edges = HoverTool(
    tooltips=[
        ("hsim", "@weight"),
        ("lsrp", "@lsrp")],
    renderers=[network_graph.edge_renderer],
    line_policy="interp")

hover_nodes = HoverTool(
    tooltips=[
        ("track","@index"),
        ("title","@node_title"),
        ("artist","@node_artist"),
        ("degree", "@degree"), 
        ("Modularity Class", "@modularity_cls")],
    renderers=[network_graph.node_renderer])

#A dd network graph to the plot
plot.renderers.append(network_graph)
plot.add_tools(hover_edges, hover_nodes)

# Some data useful for re-freshing the network after filtration
weights = [G.get_edge_data(edge[0], edge[1])["weight"] \
    for edge in list(G.edges())]
sources = [edge[0] for edge in G.edges()]
targets = [edge[1] for edge in G.edges()]

callback = CustomJS(args = dict(
    graph_setup=network_graph, weights=weights,
    sources=sources, targets=targets), code = filtration_code)
range_slider = RangeSlider(
    start=.05, end=1., value=(.05, 1.), step=.05, title="HSim Range")
range_slider.js_on_change('value', callback)

text_input_a = TextInput(value="", title="Track A:")
text_div_a = Div(width=200, height=100)
text_input_b = TextInput(value="", title="Track B:")
text_div_b = Div(width=200, height=100)

text_input_a.js_on_change("value", CustomJS(
    args=dict(div=text_div_a, meta=meta_dict),
    code=node_info_code))
text_input_b.js_on_change("value", CustomJS(
    args=dict(div=text_div_b, meta=meta_dict),
    code=node_info_code))

match_div = Div(width=200, height=100)
button = Button(label="Check match", button_type="success")
button.js_on_click(CustomJS(
    args=dict(hsim=hsim_map, div=match_div,
              track_a=text_input_a, track_b=text_input_b), code="""
var text = ''
if (track_a.value in hsim && track_b.value in hsim[track_a.value]) {
    const hval = hsim[track_a.value][track_b.value];
    text='<b>HSim:</b> ' + hval[0] + '<br>';
    text+='<b>Shared patterns:</b> ' + hval[1] + '<br>';
} else {
    text = "No harmonic match was found!";
}
div.text = text
"""))

layout = Row(plot, Column(
    range_slider, text_input_a, text_div_a,
    text_input_b, text_div_b, button, match_div))

#show(layout)
output_file(title="HSim Visualiser", filename="test.html")
save(layout)

'/Users/jacopodeberardinis/Documents/projects/harmonic-similarity/src/test.html'

In [ ]:
node_info_code = '''
    var text = "";
    if (this.value in meta) {
        text='<b>Title:</b> ' + meta[this.value]['title'] + '<br>';
        text+='<b>Artist:</b> ' + meta[this.value]['artist'] + '<br>';
        text+='<b>Link:</b> ' + '<a href=' + meta[this.value]['link'] + ', target="_blank">Listen on Soundcloud</a>';
    } else {
        text=this.value + " was not found in the graph!";
    }
    div.text = text;
'''